In [11]:
import keras
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Activation, \
    Flatten, Lambda, LSTM, RepeatVector, TimeDistributed, Reshape, \
    Conv2D, MaxPooling2D, BatchNormalization, ConvLSTM2D, Bidirectional
import numpy as np
import h5py
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import scale

In [47]:
data = h5py.File("../dataset/data_sample.hdf5", "r")
images = data["images"][:]
labels = data["labels"][:]

def scale_images(images):
    t_images = np.transpose(images, (0,3,1,2))
    rt_images = t_images.reshape(72000*48, 21, 21)
    max_per_img = np.max(rt_images.reshape(-1, 21*21), axis=1, keepdims=1)
    scaled_images = rt_images.reshape(-1, 21*21) / max_per_img
    scaled_images = scaled_images.reshape(-1, 21, 21).reshape(-1, 48, 21, 21)
    return scaled_images

def txt2digit(labels):
    dic = {'Asteroids':0, 'Constant':1, 'EmptyLigh':2, 'M33Cephei':3, 'RRLyrae':4, 'Supernova':5}
    labels_digit = np.array([dic[i] for i in labels])
    return labels_digit


def build_dataset(images, labels, seq2seq=False):
    
    train_indices = np.random.choice(np.arange(images.shape[0]), int( 0.7 * images.shape[0]))
    val_indices = list(set(np.arange(images.shape[0])) - set(train_indices))
    
    x = images
    x = np.expand_dims(x, len(x.shape))
    y = to_categorical(txt2digit(labels))
    
    if seq2seq:
        y = np.repeat(y, 48).reshape(-1, 48, 6)
    
    return x, y


x, y = build_dataset(scale_images(images), labels)

In [96]:
def init_model3(cnn_input_dim=21, cnn_output_dim=128, cnn_dropout=0.5,
               rnn_hidden_dim=128, rnn_output_dim=64, num_classes=6, rnn_dropout=0.5, timestep=48,
               bidir_mode='concat'):
    
    input_shape = (timestep, cnn_input_dim, cnn_input_dim, 1, )
    
    model = Sequential()
    
    model.add(TimeDistributed(Conv2D(48, (4,4), \
                     padding='same', activation='relu', kernel_initializer='uniform'), input_shape=input_shape))
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))
    
    model.add(TimeDistributed(Conv2D(24, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))

    model.add(TimeDistributed(Conv2D(12, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))
    
    model.add(TimeDistributed(Flatten()))

    model.add(TimeDistributed(Dense(128, activation='relu')))
    model.add(Dropout(0.25))
    
    model.add(LSTM(rnn_hidden_dim, dropout=rnn_dropout, return_sequences=True))
    model.add(LSTM(rnn_hidden_dim, dropout=rnn_dropout))
    
    model.add(Dense(6, activation='softmax'))
    
    return model

In [101]:
def init_model3_predict(cnn_input_dim=21, cnn_output_dim=128, cnn_dropout=0.5,
               rnn_hidden_dim=128, rnn_output_dim=64, num_classes=6, rnn_dropout=0.5, timestep=48,
               bidir_mode='concat'):
    
    input_shape = (1, 1, cnn_input_dim, cnn_input_dim, 1)
    
    model = Sequential()
    
    model.add(TimeDistributed(Conv2D(48, (4,4), \
                     padding='same', activation='relu', kernel_initializer='uniform'), batch_input_shape=input_shape))
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))
    
    model.add(TimeDistributed(Conv2D(24, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))

    model.add(TimeDistributed(Conv2D(12, (3,3), padding='same', activation='relu', kernel_initializer='uniform')))
    model.add(TimeDistributed(MaxPooling2D((3,3), strides=(1,1))))
    
    model.add(TimeDistributed(Flatten()))

    model.add(TimeDistributed(Dense(128, activation='relu')))
    model.add(Dropout(0.25))
    
    model.add(LSTM(rnn_hidden_dim, dropout=rnn_dropout, return_sequences=True, stateful=True))
    model.add(LSTM(rnn_hidden_dim, dropout=rnn_dropout, stateful=True))
    
    model.add(Dense(6, activation='softmax'))
    
    return model

In [118]:
model = init_model3()
model.compile(optimizer=keras.optimizers.Adam(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [119]:
model.fit(x[:100], y[:100],
              batch_size=10,
              epochs=10,
              validation_split=0.2,
              shuffle=True)

Train on 80 samples, validate on 20 samples
Epoch 1/10
80/80 [==============================] - 13s 161ms/step - loss: 1.7985 - acc: 0.1625 - val_loss: 1.7831 - val_acc: 0.2000
Epoch 2/10
80/80 [==============================] - 9s 113ms/step - loss: 1.7923 - acc: 0.2000 - val_loss: 1.7780 - val_acc: 0.1000
Epoch 3/10
80/80 [==============================] - 9s 113ms/step - loss: 1.7892 - acc: 0.1750 - val_loss: 1.7779 - val_acc: 0.1000
Epoch 4/10
80/80 [==============================] - 12s 145ms/step - loss: 1.7871 - acc: 0.2000 - val_loss: 1.7687 - val_acc: 0.1000
Epoch 5/10
80/80 [==============================] - 9s 116ms/step - loss: 1.7853 - acc: 0.2000 - val_loss: 1.7597 - val_acc: 0.1000
Epoch 6/10
80/80 [==============================] - 9s 116ms/step - loss: 1.7905 - acc: 0.1875 - val_loss: 1.7727 - val_acc: 0.1000
Epoch 7/10
80/80 [==============================] - 9s 112ms/step - loss: 1.7854 - acc: 0.2000 - val_loss: 1.7793 - val_acc: 0.1000
Epoch 8/10
80/80 [============

In [121]:
model.predict_classes(x[:100])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [98]:
# model = keras.models.load_model('../src/saved_model/model3_bdir=concat_epoch=175.h5')

In [107]:
weights = model.get_weights()

In [108]:
model_pred = init_model3_predict()

In [109]:
model_pred.set_weights(weights)

In [110]:
count = 0
for ind in range(10):
    model_pred.reset_states()
    
    for i in range(48):
        pred = model_pred.predict_classes(x[ind,i,:,:,:].reshape([1,1,21,21,1]))[0]
    print(pred, dic[labels[ind]])
    if pred == dic[labels[ind]]:
        count += 1
        
count

1 1
1 3
1 3
1 3
1 0
1 1
1 2
1 1
1 1
1 5


4

In [88]:
target_layer = model.layers[-4]
outputs = target_layer(target_layer.get_input_at(0))
m = Model(model.input, outputs)
print(m.predict(x[0,:,:,:,:].reshape(1,48,21,21,1)))

[[[0.         0.00377904 0.0051192  ... 0.         0.01647188 0.        ]
  [0.         0.02305488 0.02058041 ... 0.         0.03521    0.        ]
  [0.         0.05396858 0.05998888 ... 0.         0.10102537 0.        ]
  ...
  [0.         0.2064439  0.06123681 ... 0.08804832 0.19757879 0.        ]
  [0.         0.05900613 0.03091917 ... 0.         0.07952505 0.        ]
  [0.         0.08889082 0.03178672 ... 0.         0.11033916 0.        ]]]


In [86]:
target_layer = model_pred.layers[-4]
outputs = target_layer(target_layer.get_input_at(0))
m = Model(model_pred.input, outputs)
for i in range(48):
    pred = m.predict(x[0,i,:,:,:].reshape(1,1,21,21,1))
    
print(pred)

[[[0.00000000e+00 8.88908356e-02 3.17867287e-02 1.29932418e-01
   0.00000000e+00 0.00000000e+00 0.00000000e+00 1.20437920e-01
   0.00000000e+00 2.90362798e-02 1.31032214e-01 0.00000000e+00
   0.00000000e+00 1.72065571e-04 7.86366984e-02 0.00000000e+00
   0.00000000e+00 0.00000000e+00 6.54033646e-02 1.03466939e-02
   0.00000000e+00 0.00000000e+00 1.28941357e-01 1.09472297e-01
   0.00000000e+00 0.00000000e+00 0.00000000e+00 6.82674125e-02
   0.00000000e+00 0.00000000e+00 7.76161700e-02 0.00000000e+00
   8.35586637e-02 9.71488506e-02 0.00000000e+00 0.00000000e+00
   9.46407989e-02 0.00000000e+00 1.11906074e-01 0.00000000e+00
   1.02425069e-01 1.58406958e-01 8.27663541e-02 0.00000000e+00
   7.34841228e-02 0.00000000e+00 0.00000000e+00 0.00000000e+00
   9.28718820e-02 3.36131416e-02 0.00000000e+00 0.00000000e+00
   0.00000000e+00 9.92306471e-02 0.00000000e+00 9.84795168e-02
   4.13722023e-02 5.98364286e-02 0.00000000e+00 0.00000000e+00
   0.00000000e+00 2.48822957e-01 0.00000000e+00 9.15393

In [82]:
target_layer.get_input_at(1)

<tf.Tensor 'time_distributed_69_1/Reshape_1:0' shape=(?, 48, 128) dtype=float32>

In [48]:
model.predict_classes(x[:10])

array([1, 3, 3, 3, 0, 1, 2, 1, 1, 5])

In [55]:
x[0,0,:,:,:].reshape(1,1,21,21,1).shape

(1, 1, 21, 21, 1)

In [49]:
model.evaluate(x[:10], y[:10])

10/10 [==============================] - 1s 90ms/step


[0.009157541207969189, 1.0]

In [35]:
labels[0]

'Constant'

In [3]:
model = Model(input=input, output=net)

model.fit(...)

w = model.get_weights()
net = Dense(1)(SimpleRNN(stateful=True)(input))
model = Model(input=input, output=net)
model.set_weights(w)

'2.1.6'

In [16]:
!sudo pip3 install --upgrade tensorflow

Password:
